In [ ]:
#import all library
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import time
import schedule
import requests



In [ ]:
# assign your website to scrape , in this case amazon
web = 'https://www.amazon.com'
options = webdriver.ChromeOptions()

driver = webdriver.Chrome()
driver.get(web)

driver.implicitly_wait(5)
# select the keyword to Scrape
keyword = "wireless"
search = driver.find_element(By.ID, 'twotabsearchtextbox')
search.send_keys(keyword)
# click search button
search_button = driver.find_element(By.ID, 'nav-search-submit-button')
search_button.click()

driver.implicitly_wait(5)

In [ ]:
#we create a dataframe to collect all data scraped
product_name = []
product_asin = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []
df = pd.DataFrame(columns=['Name','price','rating','#rating' , 'link'])

In [ ]:
#this function implement the scraper function
def scrape_page(main_df):
    df=main_df.copy()
    #the item are the boxes of the product
    items = wait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
    for item in items:
        # find name
        name = item.find_element(By.XPATH, './/span[@class="a-size-medium a-color-base a-text-normal"]')
        product_name.append(name.text)

        # find price
        whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
        fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')

        if whole_price != [] and fraction_price != []:
            price = '.'.join([whole_price[0].text, fraction_price[0].text])
        else:
            price = 0
        product_price.append(price)

        # find ratings box
        ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

        # find ratings and ratings_num
        if ratings_box != []:
            ratings = ratings_box[0].get_attribute('aria-label')
            ratings_num = ratings_box[1].get_attribute('aria-label')
        else:
            ratings, ratings_num = 0, 0

        product_ratings.append(ratings)
        product_ratings_num.append(str(ratings_num))

        # find link
        boxlinks = driver.find_elements(By.XPATH, '//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
        for boxlink in boxlinks:
            link = boxlink.get_attribute('href')
        #add a line on dataframe
        new_row = {'Name':name.text, 'price':price, 'rating':ratings, '#rating':str(ratings_num) , 'link':link}
        df = df.append(new_row, ignore_index=True)
        return df



In [ ]:
#we send data as message on channel on telegram
#step:
# 1. create bot and extract token
# 2. add the bot to a new channel and extract the chat_id
def send_data():
    TOKEN = "5532274185:AAGE8CQOwV6i6VtER1xBAfhciVv4WBDSLWA"
    chat_id = "-1001774376845"

    for row in df.itertuples():
        str_link = row.link
        str_name = row.Name
        str_price = row.price
        print("🔴 ",str_price," 🔴","\n",str_name,"\n",str_link)
        text = "🔴 "+str(str_price)+" $"+" 🔴"+"%0A"+str_name+"%0A"+str_link
        url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={text}"
        #we can select the ratio of messages
        time.sleep(2)
        r = requests.get(url)
        print(r.json())

In [ ]:
#MAIN
#reset dataframe
df = pd.DataFrame(columns=['Name','price','rating','#rating' , 'link'])
#collect all data and create df with all data
df=scrape_page(df)
for i in range(2,20):
    #s-pagination-item s-pagination-next s-pagination-button s-pagination-separator
    page_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    page_button.click()
    df=scrape_page(df)
    time.sleep(2)
driver.quit()

#send data to the channel   
send_data()


In [ ]:
# to check data scraped
print(product_name)
print(product_asin)
print(product_price)
print(product_ratings)
print(product_ratings_num)
print(product_link)

In [ ]:
print(df)

DA QUI IN GIU TUTTO IL CODICE PER TESTARE

In [ ]:
#tutti i link della pagina
links = driver.find_elements(By.XPATH, '//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
l=[]
for link in links:
    value = link.get_attribute('href')
    print(value)
    l.append(value)
print("_________________________________________________________")
print(l)


In [ ]:
#legge tutte le righe del df
for row in df.itertuples():
    print(row.link)
    print(row.Index)

In [ ]:
#serve a mandare i messaggi sul canale
import time
import schedule
import requests

import requests
TOKEN = "5532274185:AAGE8CQOwV6i6VtER1xBAfhciVv4WBDSLWA"
chat_id = "-1001774376845"
text = "ciaooooo"
url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={text}"
# url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
r = requests.get(url)
print(r.json())

In [ ]:
for i in range(2,20):
    #s-pagination-item s-pagination-next s-pagination-button s-pagination-separator
    page_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    page_button.click()
    time.sleep(2)

